import pandas as pd
from IPython.display import display

In [3]:
df = pd.read_pickle('../data/data_2016_w_descs_and_census.pkl')

In [6]:
with pd.option_context("display.max_rows", 500, 'display.max_columns', 500):
    display(df.head().drop(
            ['OnTime_Status', 'OPEN_DT', 'CLOSED_DT', 'CASE_TITLE', 'SUBJECT', 'REASON', 'QUEUE',
            'Department', 'SubmittedPhoto', 'ClosedPhoto', 'fire_district', 'pwd_district', 'city_council_district',
            'police_district', 'neighborhood_services_district', 'ward', 'precinct', 'land_usage'], 
            axis=1)
    )

,CASE_ENQUIRY_ID,TARGET_DT,CASE_STATUS,CLOSURE_REASON,TYPE,Location,neighborhood,LOCATION_STREET_NAME,LOCATION_ZIPCODE,Property_Type,Property_ID,LATITUDE,LONGITUDE,Source,Geocoded_Location,case_enquiry_id,description,specific_location,title,tract_and_block_group,bedroom_total_ppl,bedroom_0,bedroom_1,bedroom_2,bedroom_3,bedroom_4,bedroom_5+,school_total,school_0_none,school_1_preschool,school_2_kindergarden,school_3_1st_grade,school_4_2nd_grade,school_5_3rd_grade,school_6_4th_grade,school_7_5th_grade,school_8_6th_grade,school_9_7th_grade,school_10_8th_grade,school_11_9th_grade,school_12_10th_grade,school_13_11th_grade,school_14_12th_grade_no_diploma,school_15_hs_diploma,school_16_ged,school_17_less_than_1_yr,school_18_some_college_no_degree,school_19_associates,school_20_bachelors,school_21_masters,school_22_professional_school,school_23_doctorate,rent_total,rent_0_100,rent_100_149,rent_150_199,rent_200_249,rent_250_299,rent_300_349,rent_350_399,rent_400_449,rent_450_499,rent_500_549,rent_550_599,rent_600_649,rent_650_699,rent_700_749,rent_750_799,rent_800_899,rent_900_999,rent_1000_1249,rent_1250_1499,rent_1500_1999,rent_2000_2499,rent_2500_2999,rent_3000_3499,rent_3500+,race_total,race_white,race_black,race_asian,race_hispanic,race_other,income_total,income_0_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000+,poverty_total_pop,poverty_pop_below_poverty_level,income_per_capita,poverty_pop_w_public_assistance,poverty_pop_w_food_stamps,poverty_pop_w_ssi,housing_total,housing_own,housing_rent,value_total,value_0_10000,value_10000_14999,value_15000_19999,value_20000_24999,value_25000_29999,value_30000_34999,value_35000_39999,value_40000_49999,value_50000_59999,value_60000_69999,value_70000_79999,value_80000_89999,value_90000_99999,value_100000_124999,value_125000_149999,value_150000_174999,value_175000_199999,value_200000_249999,value_250000_299999,value_300000_399999,value_400000_499999,value_500000_749999,value_750000_999999,value_1000000_1499999,value_1500000_1999999,value_2000000+,<1h_completion,is_description_null
13,101001867997,2016-07-27 08:43:35,Closed,Case Closed. Closed date : 2016-07-26 10:58:29...,Parking Enforcement,38 Sussex St Mission Hill MA 02120,Roxbury,38 Sussex St,2120.0,Address,132862.0,42.3360,-71.0839,Citizens Connect App,"(42.336, -71.0839)",NaN,NaN,NaN,NaN,0805001,510.0,19.0,77.0,242.0,150.0,16.0,6.0,573.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,42.0,0.0,15.0,33.0,7.0,29.0,7.0,135.0,8.0,8.0,82.0,58.0,59.0,77.0,0.0,2.0,388.0,0.0,0.0,0.0,12.0,12.0,0.0,40.0,15.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,35.0,75.0,45.0,74.0,48.0,13.0,0.0,6.0,0.0,966.0,167.0,279.0,40.0,412.0,68.0,446.0,42.0,74.0,44.0,50.0,43.0,8.0,52.0,19.0,28.0,17.0,0.0,31.0,21.0,0.0,0.0,17.0,446.0,118.0,20590,14.0,133.0,47.0,446.0,58.0,388.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,17.0,16.0,9.0,0.0,0.0,0.0,0.0,False,True
14,101001873009,NaT,Open,NaN,Animal Generic Request,107-129 Kingston St Boston MA 02111,Downtown / Financial District,107-129 Kingston St,2111.0,Address,166787.0,42.3521,-71.0596,Constituent Call,"(42.3521, -71.0596)",NaN,NaN,NaN,NaN,0701017,681.0,79.0,353.0,241.0,8.0,0.0,0.0,580.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,209.0,126.0,69.0,17.0,552.0,16.0,0.0,0.0,30.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,14.0,40.0,160.0,100.0,140.0,0.0,970.0,401.0,47.0,389.0,97.0,36.0,564.0,208.0,42.0,17.0,38.0,0.0,29.0,13.0,0.0,31.0,14.0,51.0,36.0,23.0,0.0,31.0,31.0,564.0,308.0,29542,0.0,78.0,13.0,564.0,12.0,552.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
16,101001907883,2016-09-15 08:30:00,Closed,Case Closed. Closed date : 2016-09-14 00:10:05

## Is there a 1-1 rel btwn queue and type?

In [9]:
by_type = set(df[['TYPE', 'QUEUE']].groupby('TYPE').count()['QUEUE'].tolist())

In [10]:
by_queue = set(df[['TYPE', 'QUEUE']].groupby('QUEUE').count()['TYPE'].tolist())

In [11]:
len(by_type), len(by_queue)

(143, 141)

In [12]:
by_type == by_queue

False

In [13]:
def one_one_rel_check(col1_name, col2_name, df):
    by_col1 = set(df[[col1_name, col2_name]].groupby(col1_name).count()[col2_name].tolist())
    by_col2 = set(df[[col2_name, col1_name]].groupby(col2_name).count()[col1_name].tolist())
    return by_col1 == by_col2

In [30]:
def num_distincts(lst):
    return len(set(lst))

In [37]:
def many_one_rel_check(col1_name, col2_name, df):
    """Is there a clean many-one rel?"""
    if len(df[[col1_name, col2_name]].groupby(col1_name).agg(num_distincts).drop_duplicates()) == 1:
        return True
    else:
        col1_name, col2_name = col2_name, col1_name
        if len(df[[col1_name, col2_name]].groupby(col1_name).agg(num_distincts).drop_duplicates()) == 1:
            return True
        else:
            return False

In [14]:
one_one_rel_check('TYPE', 'QUEUE', df)

False

Looks like the answer is no. Actually, now that I thnk about it, I don't care about the `QUEUE` col.

## Is there 1-1 rel btwn reason and type?

In [18]:
one_one_rel_check('TYPE', 'REASON', df)

False

In [41]:
many_one_rel_check('TYPE', 'REASON', df)

True

## Btwn subject and type?

In [19]:
one_one_rel_check('TYPE', 'SUBJECT', df)

False

In [42]:
many_one_rel_check('TYPE', 'SUBJECT', df)

True

## Btwn title and type?

In [20]:
one_one_rel_check('TYPE', 'CASE_TITLE', df)

False

Wht are the diffs?

In [24]:
col1_name = 'TYPE'
col2_name = 'CASE_TITLE'
by_col1 = df[[col1_name, col2_name]].groupby(col1_name).count()[col2_name]
by_col2 = df[[col2_name, col1_name]].groupby(col2_name).count()[col1_name]

In [25]:
by_col2

CASE_TITLE
2 lights by Arlington and Boylston are out/printed for jim k       1
2 outages in alley////printed                                      1
4 Gas Lights need mantles. - Printed EP                            1
50/50 on mall contractor light/sent                                1
50/50 replacement requerst                                         1
50/50/printed for jim k                                            1
Ababndoned Bicycle                                                 1
Abandonded Bicycle                                                 1
Abandoned Bicycle                                                771
Abandoned Buicycle                                                 1
Abandoned Building                                               149
Abandoned Lock                                                     1
Abandoned Vehicles                                              3666
Abandoned bicycle                                                  4
Abandoned bike         

In [43]:
many_one_rel_check('TYPE', 'CASE_TITLE', df)

False

## Is reason a subcateg of subject, or the other way around?

In [45]:
many_one_rel_check('SUBJECT', 'REASON', df)

False

Neither. `TYPE` is a subcategory of `SUBJECT`, and a subcategory of `REASON`, but `SUBJECT` is not a subcategory of `REASON`, or vice-versa.

Let's look at all the `SUBJECT`s.

In [46]:
df.SUBJECT.drop_duplicates()

13        Transportation - Traffic Division
14                           Animal Control
18                  Public Works Department
23                    Inspectional Services
24                      Property Management
76            Parks & Recreation Department
257                            Civil Rights
321                 Mayor's 24 Hour Hotline
327                Boston Police Department
865         Boston Water & Sewer Commission
2007                  Disability Department
75770                 Neighborhood Services
104087         Consumer Affairs & Licensing
739754                            Youthline
Name: SUBJECT, dtype: object

and for `REASON`

In [47]:
df.REASON.drop_duplicates()

13               Enforcement & Abandoned Vehicles
14                                  Animal Issues
18                                     Sanitation
23                                       Building
24                                       Graffiti
28                            Highway Maintenance
43                                        Housing
49                                      Recycling
76                      Park Maintenance & Safety
92                         Environmental Services
100             Administrative & General Requests
117                               Signs & Signals
119                               Street Cleaning
206                                        Health
209                                         Trees
224                              Code Enforcement
257                                  Call Inquiry
321                   Employee & General Comments
327                             Noise Disturbance
335                                  Notification


These are a bit too broad to use in place of `TITLE`, but may end up being useful later on.

## In conjunction w Tableau, let's look at outliers for issues that take too long

Adding `description` and `closure_removal` fields, there anecdotally seem to be reasons for the issues that take too long. Some are "administratively closed" if they're not responded to after a long time.